In [ ]:
"""
xxx

Attributes:
    | xxx (:obj:`xxx`): xxx.

Returns:
    xxx
"""

# Master project main script

In [3]:
import pyham
from Bio import SeqIO
import os

# Path to nwk file
nwk_path = "/Users/admin/phD/data/ALFSIM/KARIM/RealTree.nwk"

# The newick tree is extracted as a string
tree_str = pyham.utils.get_newick_string(nwk_path, type="nwk")

# Path to OrthoXML file
orthoxml_path = "/Users/admin/phD/data/ALFSIM/KARIM/Output/HOGs_pairs_80.orthoxml"

ham_analysis = pyham.Ham(tree_str, orthoxml_path, use_internal_name=True)

ham_analysis_no_name = pyham.Ham(tree_str, orthoxml_path, use_internal_name=False)

## Loci assignment

In [ ]:
def locus_to_id(file):
    
    '''
    This function extracts loci from a fasta file and store the information as keys in a dictionary 
    and the related protein ID as value.
    '''
    
    locus_to_id = {}
    for seq_record in SeqIO.parse(file, 'fasta'):
        locus_to_id[int(seq_record.description.split('locus: ')[-1])] = seq_record.id[:-1]
            
    return locus_to_id


def get_locus(directory):
    
    '''
    This function goes through each file of a folder to build a dictionary with loci 
    as keys and the related protein ID as values. For each entry in the dictionary, the function 
    updates the corresponding gene object by adding its locus.
    '''
    
    path = directory
    for file in os.listdir(path):
        path_to_file = path + '/' + file
        locus_dictionary = locus_to_id(path_to_file)
        
        ####### assignement only for file genome 
        
        ## genome_name = get_name_from_fasta_file(file)
        
        # each gene assign else none
        
        for entry in locus_dictionary:
            gene = ham_analysis_no_name.get_genes_by_external_id(locus_dictionary[entry]) ## more than 1 ?
            for element in gene:
                element.locus = entry
                
        once genome/file process genome.locus_Assignement = True
                
# sanity check

look if all ectant have locus_Assignement == True

In [ ]:
get_locus('/Users/admin/phD/data/ALFSIM/KARIM/DB')

In [ ]:
'''
Function to get a gene object nowing a specific locus. 
This function should be used to assign neighbors in this
new version of the code.
'''

def get_gene_by_locus(genome, locus):
    for gene in genome.genes:
        if gene.locus == locus:
            return gene  
    return None

## Neighbors assignment

In [ ]:
def assign_neighbors(genome):
    
    if isinstance(genome, pyham.ExtantGenome):

        for extant_gene in genome.genes: ## if  extant_gene.left set by default to None no need to try catch. 
            
            extant_gene.left = get_gene_by_locus(genome, extant_gene.locus-1)
            extant_gene.right = get_gene_by_locus(genome, extant_gene.locus+1)
            
                        
    if isinstance(genome, pyham.AncestralGenome):
    
        '''
        If not all children have the same neighbors, we stop.
        If all children have the same neighbors, we assign.
        Once the synteny block reconstruction have been made, plot of the synteny size blocks
        '''
    
        for ancestral_gene in genome.genes: # For each HOG in the ancestral genome...
                        
            left_parent = set() 
            right_parent = set() 
            
            for child in ancestral_gene.children:
                
                try: ## why try ? --> use if for child.left.parent
                    left_parent.add(child.left.parent)
                except:
                    left_parent.add(None)
                
                try: ## why try ? --> use if for child.left.parent
                    right_parent.add(child.right.parent)
                except:
                    right_parent.add(None)
            
            if len(left_parent) == 1: # If all left neighbors have the same parent
                ancestral_gene.left = list(left_parent)[0] # Extraction of the parent from the set
            elif None in left_parent or len(left_parent) > 1: # If all children have no left neighbors
                ancestral_gene.left = None 
            
            if len(right_parent) == 1: # If all right neighbors have the same parent
                ancestral_gene.right = list(right_parent)[0] # Extraction of the parent from the set
            elif None in right_parent or len(right_parent) > 1: # If all children have no right neighbors
                ancestral_gene.right = None
                

## Synteny block reconstruction

In [ ]:
def get_synteny_blocks(genome):
    syn_blocks = []
    seen = []
    gene_collection = genome.genes
    left = True
    right = True
    tmp = []
    
    for abstract_gene in gene_collection:
        if abstract_gene not in seen: # seen is a list to keep track of already processed gene.
            seen.append(abstract_gene)
            right_gene = abstract_gene ## ?? current_right
            left_gene = abstract_gene ## ??
            left = True # When False, the growing of a synteny block is blocked leftward
            right = True # When False, the growing of a synteny block is blocked rightward
            tmp = [abstract_gene] # Temporary list for each syntenic block
            
            ## split left and right maybe
                    
            while left or right:
                if right:
                    if right_gene.right != None: # For the first iteration, right_gene is the abstract gene itself.
                        tmp.append(right_gene.right) # The syntenic block is growing rightward.
                        seen.append(right_gene.right)
                        right_gene = right_gene.right # The following gene to be investigated rightward for the next iteration is set up
                    elif right_gene.right == None:
                        right = False # The growing rightward is ended.
                
                if left:
                    if left_gene.left != None:
                        tmp.insert(0, left_gene.left) # The gene is append at the begining of the list
                        seen.append(left_gene.left)
                        left_gene = left_gene.left # The following gene to be investigated leftward for the next iteration is set up
                    else:
                        left = False
            
            syn_blocks.append(tmp) # When both left and right variables are set to False, the synteny block (tmp) is added to the list syn_blocks
            
        else:
            pass # If the abstract gene has already been investigated, it is not considered.
    
    genome.synteny = syn_blocks

'''
By adding genes at both extremities of a synteny block (tmp), the gene order is maintained.
Each genome is assigned with a new attribute (synteny), that is a list of all synteny block reconstructed.
len(syn_blocks) return the amount of synteny block reconstructed.
'''

## Processing
The tree is traversed and the function created are applied for each node.

In [ ]:

number_of_blocks = []

for node in ham_analysis_no_name.taxonomy.tree.traverse(): ## make sure .traverse('post_fix) --> bottom up
    
    assign_neighbors(node.genome)
    get_synteny_blocks(node.genome)
    
    number_of_blocks.append(len(node.genome.synteny))


## Plot

In [ ]:
import matplotlib.pyplot as plt
plt.hist(number_of_blocks, 50)
plt.xlabel('Synteny blocks')
plt.ylabel('Genome count')
plt.title('Histogram of synteny blocks')
plt.show()

Regarding the first bar, 20 genomes possess one single synteny block, which correspond to extant genomes.
The other bars are the ancestral genomes.

In [ ]:
from ete3 import Tree, TreeStyle, TextFace

t = ham_analysis_no_name.taxonomy.tree

# Basic tree style
ts = TreeStyle()
ts.show_leaf_name = True

for node in t:
    node.add_face(TextFace("[1000:3],[2000:4],[10000:1]"), column=0, position = "branch-right")
t.show(tree_style=ts)

In [ ]:


from ete3 import TreeStyle, TextFace, NodeStyle, BarChartFace

treeCopy = ham_analysis_no_name.taxonomy.tree.copy(method="newick")

pcross each genome genome.hist_value et genome.hist_labels

## genome.synteny_block = [[1000gene],[1000gene],200,300] -> hist_data value = [1,1,2] + label = [200,300,1000]

max_block = max(all genome -> value )

def _layout(node):
    
    def _add_face(name_feature, value_feature, cnum=1, pos="branch-right"):
        node.add_face(TextFace("{}: {}".format(name_feature, value_feature)), column=cnum, position=pos)

        _add_face("#genes", node.nbr_genes, cnum=cNbr, pos=posNbr)

        if node.retained is not None:
            _add_face("#Retained", node.retained, cnum=cAttr, pos=posAtt)

        if node.dupl is not None:
            _add_face("#Duplicated", node.dupl, cnum=cAttr, pos=posAtt)

        if node.gain is not None:
            _add_face("#Novel", node.gain, cnum=cAttr, pos=posAtt)

        if node.lost is not None:
            _add_face("#Lost", node.lost, cnum=cAttr, pos=posAtt)
            
    hist_data for this node.genome
    value = [1,1,2]
    _label = ['200','300','1000']
    w_plot = 10 * len(_label)
    
    if node.is_leaf():
        pos = "branch-right"
    else:
        pos = "branch-top"     
    
    node.add_face(BarChartFace(values, deviations=None, width=w_plot, height=25, colors=_color_scheme, labels=_label, min_value=0, max_value=max_block, label_fsize=6, scale_fsize=6),column=1, position = pos)
    
ts = TreeStyle()
ts.layout_fn = _layout
ts.legend.add_face(BarChartFace(_values_legend, deviations=None, width=w_legend, height=25, colors=_color_scheme, labels=_label_legend, min_value=0, max_value=max_genes, label_fsize=6, scale_fsize=6),column=0)
ts.legend_position = 3

treeCopy.render(output,tree_style=ts)
